### ***Data preparation for Traffy fondue dataset***
เตรียมข้อมูลและทำ data cleansing สำหรับข้อมูล bangkok_traffy.csv เพื่อสร้างโมเดลทำนายการแก้ไขปัญหา ช้า/ทันเวลา (late / on-time)

### 1) Examining the Data Set


In [3]:
import pandas as pd
import numpy as np
from sklearn import set_config

set_config(transform_output="pandas")

In [4]:
df = pd.read_csv('../data/bangkok_traffy.csv')

ตอนนี้เรารู้แล้วว่าชุดข้อมูลมีทั้งหมด 787,026 แถว แต่สำหรับโปรเจกต์นี้เราจะใช้เพียง 300,000 แถวเท่านั้น ปริมาณเท่านี้เพียงพอที่จะยังคงรักษาการกระจายของฟีเจอร์ให้สมจริง และยังช่วยให้ขั้นตอนการเทรนโมเดลไม่หนักจนเกินไปด้วย, แต่การจะเลือกมาแค่ 300k rows แรกก็อาจจะทำให้ data biased เราจึงเลือกใช้การ ***stratified sampling*** ตาม timestamp เพราะเรื่องที่เราต้องการจะ predict นั้น ***มีพฤติกรรมเปลี่ยนไปตามเวลา*** ดังนั้นเราจึงไม่ควรมีแค่ row ที่กระจุกอยู่ในช่วงใดช่วงนึงมากจนเกินไป

In [5]:
# แปลง timestamp เป็น datetime
df["timestamp"] = pd.to_datetime(
    df["timestamp"].astype(str).str.strip(),
    errors="coerce",     # แปลงไม่ได้ให้เป็น NaT
    utc=True,            # ใส่ utc ไปเลยจะได้เป็นมาตรฐาน
)
print(df["timestamp"].dtype)

datetime64[ns, UTC]


Stratified sampling

In [6]:
df = df.dropna(subset=["timestamp"])
df["month"] = df["timestamp"].dt.to_period("M")

# stratified sampling
target_n = 300000
sample_frac = target_n / len(df)
sampled_df = (
    df.groupby("month")
      .apply(lambda x: x.sample(frac=sample_frac, random_state=42))
      .reset_index(drop=True)
)
print(len(sampled_df), "rows in stratified sample")

/tmp/ipykernel_39756/3881669325.py:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df["month"] = df["timestamp"].dt.to_period("M")
/tmp/ipykernel_39756/3881669325.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(frac=sample_frac, random_state=42))


300002 rows in stratified sample


In [7]:
sampled_df.head()

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,month
0,2021-FYJTFP,{ความสะอาด},เขตบางซื่อ,ขยะเยอะ,https://storage.googleapis.com/traffy_public_b...,NaN,"100.53084,13.81865",12/14 ถนน กรุงเทพ- นนทบุรี แขวง บางซื่อ เขตบาง...,NaN,NaN,กรุงเทพมหานคร,2021-09-03 12:51:09.453003+00:00,เสร็จสิ้น,NaN,0,2022-06-04 15:34:14.609206+00,2021-09
1,2021-4D9Y98,{},"เขตลาดพร้าว,การไฟฟ้านครหลวง เขตนวลจันทร์",หน้าปากซอย ลาดพร้าววังหิน26,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.59131,13.80910",17/73 17/73 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขต...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-13 05:53:36.861064+00:00,เสร็จสิ้น,NaN,0,2023-03-14 12:09:14.947437+00,2021-12
2,2021-8BTWZB,{ท่อระบายน้ำ},"เขตประเวศ,ฝ่ายโยธา เขตประเวศ",ขอแจ้งเรื่องท่อระบายน้ำบนถนนในซอยเสียหาย เป็นร...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.65440,13.68158",70 ซอย เฉลิมพระเกียรติ ร. 9 แขวง หนองบอน เขต ป...,หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-22 10:15:33.294829+00:00,เสร็จสิ้น,5.0,0,2022-06-20 13:12:04.99444+00,2021-12
3,2021-DVEWYM,"{น้ำท่วม,ถนน}","เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว",ซอยลาดพร้าววังหิน 75 ถนนลาดพร้าววังหิน แขวงลาด...,https://storage.googleapis.com/traffy_public_b...,NaN,"100.59165,13.82280",702 ถ. ลาดพร้าววังหิน แขวงลาดพร้าว เขตลาดพร้าว...,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 12:29:08.408763+00:00,เสร็จสิ้น,5.0,0,2022-08-12 07:18:44.884945+00,2021-12
4,2022-7ZTKJV,"{ถนน,ทางเท้า}","เขตสาทร,ฝ่ายโยธา เขตสาทร",บริเวณนราธิวาส แยกถนนจันทน์ ใกล้สวนสาธารณะช่อ...,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.53764,13.70716",แยกจันทน์-นราธิวาส ถนน จันทน์ แขวง ทุ่งมหาเมฆ ...,ทุ่งมหาเมฆ,สาทร,กรุงเทพมหานคร,2022-01-02 10:53:25.580723+00:00,เสร็จสิ้น,NaN,0,2022-06-08 05:46:12.776594+00,2022-01


In [8]:
sampled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300002 entries, 0 to 300001
Data columns (total 17 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   ticket_id      296576 non-null  object             
 1   type           299971 non-null  object             
 2   organization   299791 non-null  object             
 3   comment        296576 non-null  object             
 4   photo          299966 non-null  object             
 5   photo_after    244578 non-null  object             
 6   coords         300002 non-null  object             
 7   address        296576 non-null  object             
 8   subdistrict    299805 non-null  object             
 9   district       299806 non-null  object             
 10  province       299936 non-null  object             
 11  timestamp      300002 non-null  datetime64[ns, UTC]
 12  state          300002 non-null  object             
 13  star           104515 non-nul

In [9]:
sampled_df.describe(include='all')

,ticket_id,type,organization,comment,photo,photo_after,coords,address,subdistrict,district,province,timestamp,state,star,count_reopen,last_activity,month
count,296576,299971,299791,296576,299966,244578,300002,296576,299805,299806,299936,300002,300002,104515.000000,300002.000000,300002,300002
unique,296576,9781,38495,268941,296532,244578,270061,177541,283,103,49,NaN,3,NaN,NaN,300002,39
top,2025-6426RQ,{},"เขตปทุมวัน,ฝ่ายเทศกิจ เขตปทุมวัน",จอดรถบนทางเท้า,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,"100.54808,13.85436",ซอย งามวงศ์วาน 30 ถนน งามวงศ์วาน แขวง ทุ่งสองห...,สวนหลวง,จตุจักร,กรุงเทพมหานคร,NaN,เสร็จสิ้น,NaN,NaN,2025-01-13 04:12:58.825873+00,2022-06
freq,1,43934,1996,576,3426,1,758,814,7529,15983,251640,NaN,246301,NaN,NaN,1,23448
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-09-19 03:37:12.344372736+00:00,NaN,3.868344,0.122739,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-09-03 12:51:09.453003+00:00,NaN,1.000000,0.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-01-13 02:10:01.398219776+00:00,NaN,3.000000,0.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-10-06 09:54:33.362564352+00:00,NaN,5.000000,0.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2024-06-05 13:05:09.504759296+00:00,NaN,5.000000,0.000000,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-01-16 02:53:27.583947+00:00,NaN,5.000000,100.000000,NaN,NaN


In [10]:
df = sampled_df

### 2) Narrowing down columns

##### Investigating at Province column

In [11]:
df["province"].value_counts()

province
กรุงเทพมหานคร                    251640
จังหวัดกรุงเทพมหานคร              47859
จังหวัดLac                          103
นนทบุรี                              75
สมุทรปราการ                          67
ปทุมธานี                             30
จังหวัดนนทบุรี                       24
จังหวัดสมุทรปราการ                   16
นครราชสีมา                           13
จังหวัดจังหวัด กรุงเทพมหานคร         10
จังหวัดสมุทรสาคร                      9
จังหวัดปทุมธานี                       9
สมุทรสาคร                             8
จังหวัดBorno                          8
นครปฐม                                6
ภูเก็ต                                5
จังหวัดเพชรบุรี                       5
จังหวัดนครปฐม                         4
จังหวัดBangkok                        3
จังหวัดราชบุรี                        3
จังหวัดTillabéri                      3
ชลบุรี                                3
ยโสธร                                 2
จังหวัดพระนครศรีอยุธยา                2
จังหวัดฉะเชิงเทรา              

จากที่ได้ investigate คอลัมน์ province มีปัญหาเรื่องการสะกดไม่สม่ำเสมอ (e.g., “กรุงเทพมหานคร”, “จังหวัดกรุงเทพมหานคร”)
มีค่าที่ผิดปกติ เช่น จังหวัดต่างประเทศ หรือจังหวัดที่สะกดผิด รวมถึง distribution ที่กระจุกตัวมากกว่า 99% อยู่ในเขตกรุงเทพฯ (flat) ทำให้เป็นฟีเจอร์ที่ไม่มีความหมายเชิงพยากรณ์ จึงไม่เหมาะสำหรับใช้เป็น input ของโมเดล ML

In [12]:
df = df.drop(columns=["province"], errors="ignore") # drop column province ไปเลย

##### Other columns that need to be drop

In [13]:
cols_to_drop = [
    "ticket_id",
    "photo",
    "photo_after",   
    "address",
    "subdistrict",
    "province",
    "state",          # leakage
]
df = df.drop(columns=cols_to_drop, errors="ignore")

- ticket_id -> เป็น unique_id ไม่มีความหมายเชิงพฤติกรรม
- photo,photo_after -> เราไม่ทำ image processing อยู่แล้ว และ photo_after จะ target leak เต็มๆ
- address -> high cardinality และใช้พวก district/subdistrict แทนได้ 
- subdistrict -> high cardinality อาจทำให้ ml overfit และใช้ district แทนได้

### 3) Preparing the Features for Machine Learning

District

In [14]:
df["district"] = df["district"].str.strip()
# ลบ whitespace เพื่อเตรียมตัวนำไปทำ one-hot vector ทีหลัง

Coords

In [15]:
coords_split = df["coords"].str.split(",", expand=True)
df["lat"] = pd.to_numeric(coords_split[0], errors="coerce")
df["lon"] = pd.to_numeric(coords_split[1], errors="coerce")
df = df.drop(columns=["coords"])
# แบ่งออกเป็น column latitude แล้ว longtitude

Timestamp

In [16]:
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce", utc=True)
df["hour"] = df["timestamp"].dt.hour
df["dayofweek"] = df["timestamp"].dt.dayofweek
df["month"] = df["timestamp"].dt.month
df["year"] = df["timestamp"].dt.year
# แปลงให้เป็น features

Last_activity

In [17]:
# นำมาสร้าง target 
df["last_activity"] = pd.to_datetime(df["last_activity"], errors="coerce", utc=True)
df["resolution_time_hours"] = (df["last_activity"] - df["timestamp"]).dt.total_seconds()/3600
# แล้วค่อย drop ทิ้ง
df = df.drop(columns=["last_activity"], errors="ignore")

# สร้าง column is_late ขึ้นมาโดยกำหนดให้เกิน 7 วัน แปลว่าช้า

# ตั้ง threshold (หน่วย: ชั่วโมง)
late_threshold = 168  # 7 วัน
# สร้างคอลัมน์ is_late (1 = ช้า, 0 = ทันเวลา)
df["is_late"] = (df["resolution_time_hours"] > late_threshold).astype(int)

df["is_late"].value_counts(normalize=True)


is_late
1    0.529053
0    0.470947
Name: proportion, dtype: float64

จะเห็นได้ว่า class มัน balance และไม่ oversample หรือ undersample จากการเลือก threshold เป็น 7 วัน

Star

In [18]:
na_count = df["star"].isna().sum()
print("Number of NA in star:", na_count)
na_ratio = df["star"].isna().mean() * 100
print(f"Percentage NA in star: {na_ratio:.2f}%")

Number of NA in star: 195487
Percentage NA in star: 65.16%


จะเห็นได้ว่า star มี na เยอะแต่เราตัดสินใจไม่ drop เนื่องจากมันมีความสำคัญต่อการ predict ว่า ticket ไหนแก้ไข late หรือ on-time <br>
solution : fill ด้วย 0 (แปลว่าไม่ได้ให้คะแนน)

In [19]:
df["star"] = df["star"].fillna(0)

na_count = df["star"].isna().sum()
print("Number of NA in star:", na_count)
na_ratio = df["star"].isna().mean() * 100
print(f"Percentage NA in star: {na_ratio:.2f}%")

Number of NA in star: 0
Percentage NA in star: 0.00%


Count_reopen

In [20]:
df["count_reopen"] = df["count_reopen"].fillna(0)

Organization

In [21]:
df["organization"] = df["organization"].fillna("Unknown")

In [22]:
import os
out_dir = '../data'
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, 'bangkok_traffy_clean.csv')
df.to_csv(out_path, index=False, encoding='utf-8-sig')
print('Saved cleaned traffy to', out_path)

Saved cleaned traffy to ../data/bangkok_traffy_clean.csv
